In [3]:
import pandas as pd

#### Часть, где все описания опускаются в нижний регистр, удаляются теги, nan значения. Затем датасеты сохраняются в файлы, чтобы снова их не пересчитывать

In [19]:
# Файл train.csv содержит вакансии, которые имеют целевую переменную
train = pd.read_csv('train.csv', header=0, delimiter="\t", quoting=3)

#Файл test.csv содержит тексты, по которым нужно выдать предсказания
test = pd.read_csv('test.csv', header=0,  delimiter="\t", quoting=3)


In [145]:
other = pd.read_csv('other.csv', header=0,  delimiter="\t", quoting=3)

In [21]:
#в трейне у каких-то вакансий не доставало таргета, поэтому я их убрала
traindataset = train.copy()
traindataset = traindataset[~traindataset['target'].isnull()]

In [22]:
#в тесте были вакансии только с id, но видимо без описаний и названий. Убираем их
testdataset = test.copy()
testdataset = testdataset[~testdataset['description'].isnull()]

In [25]:
from bs4 import BeautifulSoup
import re
def without_tags(s):
    return BeautifulSoup(s, 'lxml').text.lower()

In [26]:
# удаляем теги
# у меня это длится всё 7 минут примерно для каждого датасета.
traindataset['description']=traindataset['description'].map(without_tags)

In [27]:
%%time
testdataset['description']=testdataset['description'].map(without_tags)

Wall time: 7min 11s


In [176]:
%%time
other2=other.copy()
other2 = other2[~other2['description'].isnull()]
other2['description']=other2['description'].map(without_tags)

Wall time: 30min 34s


In [177]:
filename = 'other_without_tags_low.csv'
other2.to_csv(filename,index=False, sep="\t")
print('Saved file: ' + filename)

Saved file: other_without_tags_low.csv


In [134]:
filename = 'train_without_tags_low.csv'
traindataset.to_csv(filename,index=False, sep="\t")
print('Saved file: ' + filename)

Saved file: train_without_tags_low.csv


In [135]:
filename = 'test_without_tags_low.csv'
testdataset.to_csv(filename,index=False, sep="\t")
print('Saved file: ' + filename)

Saved file: test_without_tags_low.csv


#### Окончание обработки датасетов и создания файлов

In [4]:
train_tags = pd.read_csv('train_without_tags_low.csv', header=0, delimiter="\t", quoting=3)

In [5]:
train_tags.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,в новый магазин york (хозтовары) приглашаем на...,1.0
1,1,Наладчик станков и манипуляторов с ПУ,обязанности:работа на токарных станках с чпу t...,0.0
2,2,Разработчик С++ (Криптограф),требования: опыт программирования на с++ знан...,0.0
3,3,Фрезеровщик,условия: на работу вахтовым методом в г. калу...,0.0
4,4,Мерчендайзер/продавец-консультант,компания палладиум стандарт - признанный лидер...,1.0


In [6]:
test_tags = pd.read_csv('test_without_tags_low.csv', header=0, delimiter="\t", quoting=3)

In [7]:
test_tags.head(10)

,id,name,description
0,200000,Дизайнер-консультант мебели,"обязанности: работа с клиентом в салоне,выезд..."
1,200001,Продавец-консультант (ТЦ на Пушкина),обязанности: ∙ консультирование покупателей по...
2,200002,Менеджер по продажам,торговый дом «форт» это ведущая компания петер...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,"""требуются продавцы консультанты в магазин жен..."
4,200004,Специалист по охране труда,обязанности: осуществление контроля по соблю...
5,200005,Эксперт по обеспечению качества при сооружении...,"обязанности: управление несоответствиями, в..."
6,200006,Торговый представитель (Арзамас),обязанности: ведение и развитие существующей ...
7,200007,Заместитель генерального директора по производ...,"""обязанности: доработка качества, в первую оч..."
8,200008,Backend Rust developer,storiqa - это площадка для торговли физическим...
9,200009,Дизайнер-конструктор 3D,"""""""обязанности: адаптация дизайнов визуализац..."


In [8]:
other_tags = pd.read_csv('other_without_tags_low.csv', header=0, delimiter="\t", quoting=3)

In [9]:
other_tags.head()

,name,description
0,Специалист научно-производственного отдела,"образование - среднее специальное, высшее спец..."
1,Оператор по отгрузке товара в 1С (ТЗ),обязанности: оформление отгрузок по заявкам о...
2,Менеджер по персоналу,"обязанности: подбор персонала (специалисты, р..."
3,Ведущий бухгалтер по учёту заработной платы,"обязанности: начисление заработной платы, ра..."
4,Инженер-расчетчик в строительный отдел,требования: умение выполнять расчеты в програ...


In [10]:
import re
def text_to_wordlist(text):
    allwords = []
    text = re.sub('[^a-zа-яё]', ' ', text)
    words = text.split()
    for word in words:
        if len(word)>=5: #включаем слова больше 5 символов
            allwords.append(word)
    return allwords


In [11]:
# дальше мы формируем словарь для модели из теста,трейна,доп. файла
# Элементами массивов являются списки слов каждой статьи.
ArticleCorpusTrain = []
for review in train_tags["description"]:
    ArticleCorpusTrain.append(text_to_wordlist(review))

In [12]:
%%time
ArticleCorpusTest = []
for review in test_tags["description"]:
    ArticleCorpusTest.append(text_to_wordlist(review))

Wall time: 36.6 s


In [13]:
%%time
ArticleCorpusOther = []
for review in other_tags["description"]:
    ArticleCorpusOther.append(text_to_wordlist(review))

Wall time: 3min 7s


In [17]:
ArticleCorpus=ArticleCorpusOther+ArticleCorpusTest+ArticleCorpusTrain

In [18]:
%%time
#тренеруем модель на все слова

from gensim.models.word2vec import Word2Vec 

num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(ArticleCorpus, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

D:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Wall time: 28min 22s


In [19]:
model.init_sims(replace=True)

In [23]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [24]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [25]:
#подсчет векторов. Данный вектор строился около 2 часов на самом деле, а не сутки
%%time
train_vecs = texts_to_vecs(ArticleCorpusTrain, model, num_features)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Wall time: 23h 18min 24s


In [26]:
%%time
test_vecs = texts_to_vecs(ArticleCorpusTest, model, num_features)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


Wall time: 2h 28min 43s


In [24]:
len(train_tags["target"])

199933

In [27]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train_tags["target"])


In [28]:
predict_proba = forest.predict_proba(test_vecs)

In [29]:
submission = pd.DataFrame({'id':test_tags['id'],'target':predict_proba[:,1]})
submission.head(10)

,id,target
0,200000,0.8200
1,200001,1.0000
2,200002,0.9900
3,200003,0.9900
4,200004,0.0175
5,200005,0.0000
6,200006,1.0000
7,200007,0.2900
8,200008,0.0000
9,200009,0.0100


In [31]:
filename = 'targets_2.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

Saved file: targets_2.csv
